<a href="https://colab.research.google.com/github/mauney/DS-Unit-2-Kaggle-Challenge/blob/master/module1/assignment_kaggle_challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 1

## Assignment
- [X] Do train/validate/test split with the Tanzania Waterpumps data.
- [X] Define a function to wrangle train, validate, and test sets in the same way. Clean outliers and engineer features. (For example, [what other columns have zeros and shouldn't?](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values) What other columns are duplicates, or nearly duplicates? Can you extract the year from date_recorded? Can you engineer new features, such as the number of years from waterpump construction to waterpump inspection?)
- [X] Select features. Use a scikit-learn pipeline to encode categoricals, impute missing values, and fit a decision tree classifier.
- [ ] Get your validation accuracy score.
- [ ] Get and plot your feature importances.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

### Reading

- A Visual Introduction to Machine Learning
  - [Part 1: A Decision Tree](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)
  - [Part 2: Bias and Variance](http://www.r2d3.us/visual-intro-to-machine-learning-part-2/)
- [Decision Trees: Advantages & Disadvantages](https://christophm.github.io/interpretable-ml-book/tree.html#advantages-2)
- [How a Russian mathematician constructed a decision tree — by hand — to solve a medical problem](http://fastml.com/how-a-russian-mathematician-constructed-a-decision-tree-by-hand-to-solve-a-medical-problem/)
- [How decision trees work](https://brohrer.github.io/how_decision_trees_work.html)
- [Let’s Write a Decision Tree Classifier from Scratch](https://www.youtube.com/watch?v=LDRbO9a6XPU) — _Don’t worry about understanding the code, just get introduced to the concepts. This 10 minute video has excellent diagrams and explanations._
- [Random Forests for Complete Beginners: The definitive guide to Random Forests and Decision Trees](https://victorzhou.com/blog/intro-to-random-forests/)


### Doing
- [ ] Add your own stretch goal(s) !
- [ ] Try other [scikit-learn imputers](https://scikit-learn.org/stable/modules/impute.html).
- [ ] Try other [scikit-learn scalers](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Make exploratory visualizations and share on Slack.


#### Exploratory visualizations

Visualize the relationships between feature(s) and target. I recommend you do this with your training set, after splitting your data. 

For this problem, you may want to create a new column to represent the target as a number, 0 or 1. For example:

```python
train['functional'] = (train['status_group']=='functional').astype(int)
```



You can try [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html) for features with reasonably few unique values. (With too many unique values, the plot is unreadable.)

- Categorical features. (If there are too many unique values, you can replace less frequent values with "OTHER.")
- Numeric features. (If there are too many unique values, you can [bin with pandas cut / qcut functions](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html?highlight=qcut#discretization-and-quantiling).)

You can try [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html) with numeric features. For this problem, you may want to use the parameter `logistic=True`

You do _not_ need to use Seaborn, but it's nice because it includes confidence intervals to visualize uncertainty.

#### High-cardinality categoricals

This code from a previous assignment demonstrates how to replace less frequent values with 'OTHER'

```python
# Reduce cardinality for NEIGHBORHOOD feature ...

# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
```



In [0]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [0]:
# Split train into train and val
train, val = train_test_split(train, train_size=0.8, test_size = 0.2,
                              stratify=train['status_group'], random_state=42)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [0]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
43360,72938,0.0,2011-07-27,NaN,0,NaN,33.542898,-9.174777,Kwa Mzee Noa,0,Lake Nyasa,Mpandapanda,Mbeya,12,4,Rungwe,Kiwira,0,True,GeoData Consultants Ltd,VWC,K,NaN,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,functional
7263,65358,500.0,2011-03-23,Rc Church,2049,ACRA,34.665760,-9.308548,Kwa Yasinta Ng'Ande,0,Rufiji,Kitichi,Iringa,11,4,Njombe,Imalinyi,175,True,GeoData Consultants Ltd,WUA,Tove Mtwango gravity Scheme,True,2008,gravity,gravity,gravity,wua,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
2486,469,25.0,2011-03-07,Donor,290,Do,38.238568,-6.179919,Kwasungwini,0,Wami / Ruvu,Kwedigongo,Pwani,6,1,Bagamoyo,Mbwewe,2300,True,GeoData Consultants Ltd,VWC,NaN,False,2010,india mark ii,india mark ii,handpump,vwc,user-group,pay per bucket,per bucket,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional
313,1298,0.0,2011-07-31,Government Of Tanzania,0,DWE,30.716727,-1.289055,Kwajovin 2,0,Lake Victoria,Kihanga,Kagera,18,1,Karagwe,Isingiro,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,other,other,other,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,non functional
52726,27001,0.0,2011-03-10,Water,0,Gove,35.389331,-6.399942,Chama,0,Internal,Mtakuj,Dodoma,1,6,Bahi,Nondwa,0,True,GeoData Consultants Ltd,VWC,Zeje,True,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,functional


## wrangle train, validate, and test sets in the same way

In [0]:
import numpy as np

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['longitude', 'latitude']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)

    # drop features for now - maybe investigate later
    drop_for_now = ['funder', 'installer', 'scheme_name', 'subvillage', 'ward', 'wpt_name']       
    # drop features for reasons
    columns_to_drop = ['id', 'recorded_by', 'quality_group', 'quantity_group',
                       'date_recorded', 'extraction_type_class',
                       'extraction_type_group', 'num_private', 'payment_type',
                       'source_class', 'source_type', 'waterpoint_type_group'] + drop_for_now
    X = X.drop(columns=columns_to_drop, axis='columns')

    # combine 'other' and 'unkown' for source feature
    X['source'] = X['source'].replace(to_replace='unkown', value='other')

    # convert zeros to nan for imputing
    convert_zeros = ['amount_tsh', 'construction_year', 'district_code', 'population']
    for feature in convert_zeros:
      X[feature] = X[feature].replace(to_replace=0, value=np.nan)

    # most zero and sub-zero gps_heights are probably nan - we willl impute
    X.loc[X['gps_height'] <= 0, 'gps_height'] = np.nan
    
    # return the wrangled dataframe
    return X


train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
# Arrange data into X features matrix and y target vector
target = 'status_group'
X_train = train.drop(target, axis='columns')
y_train = train[target]
X_val = val.drop(target, axis='columns')
y_val = val[target]
X_test = test

In [0]:
train.columns

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'basin', 'region',
       'region_code', 'district_code', 'lga', 'population', 'public_meeting',
       'scheme_management', 'permit', 'construction_year', 'extraction_type',
       'management', 'management_group', 'payment', 'water_quality',
       'quantity', 'source', 'waterpoint_type', 'status_group'],
      dtype='object')

In [0]:
X_train_copy = X_train.copy()
X_val_copy = X_val.copy()

## Use scikit-learn pipelines

In [0]:
%%time
import category_encoders as ce
# explicitly require this experimental feature
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

encode_features = ['basin', 'region', 'region_code', 'district_code', 'lga',
       'scheme_management', 'extraction_type', 'management', 'management_group',
       'payment', 'water_quality', 'quantity', 'source', 'waterpoint_type']

pipeline = make_pipeline(
    ce.OrdinalEncoder(cols=encode_features, handle_missing='return_nan', handle_unknown='return_nan'),
    # ce.TargetEncoder(cols=encode_features, handle_missing='return_nan', handle_unknown='return_nan'),
    SimpleImputer(),
    # IterativeImputer(sample_posterior=True, random_state=42),
    DecisionTreeClassifier(min_samples_leaf=250, max_depth=10, random_state=42)
)

# Fit on train
pipeline.fit(X_train, y_train)

# Score on val
print('Train Accuracy', pipeline.score(X_train, y_train))
print('Validation Accuracy', pipeline.score(X_val, y_val))

# Predict on test
y_pred = pipeline.predict(X_test)



Train Accuracy 0.733459595959596
Validation Accuracy 0.7298821548821549
CPU times: user 1.54 s, sys: 118 ms, total: 1.66 s
Wall time: 1.67 s


In [0]:
X_train_copy.isnull().sum()

amount_tsh           33265
gps_height           17507
longitude             1442
latitude              1442
basin                    0
region                   0
region_code              0
district_code           19
lga                      0
population           17066
public_meeting        2644
scheme_management     3128
permit                2443
construction_year    16517
extraction_type          0
management               0
management_group         0
payment                  0
water_quality            0
quantity                 0
source                   0
waterpoint_type          0
dtype: int64

In [0]:
X_train.isnull().sum()

amount_tsh           33265
gps_height           17507
longitude             1442
latitude              1442
basin                    0
region                   0
region_code              0
district_code           19
lga                      0
population           17066
public_meeting        2644
scheme_management     3128
permit                2443
construction_year    16517
extraction_type          0
management               0
management_group         0
payment                  0
water_quality            0
quantity                 0
source                   0
waterpoint_type          0
dtype: int64

In [0]:
pipeline.steps[1]

SyntaxError: ignored

In [0]:
# Write submission file
# submission = sample_submission.copy()
# submission['status_group'] = y_pred
# submission.to_csv('submission-03.csv', index=False)

In [0]:
%%time
import category_encoders as ce
# explicitly require this experimental feature
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

encode_features = ['basin', 'region', 'region_code', 'district_code', 'lga',
       'scheme_management', 'extraction_type', 'management', 'management_group',
       'payment', 'water_quality', 'quantity', 'source', 'waterpoint_type']

pipeline = make_pipeline(
    ce.OrdinalEncoder(cols=encode_features, handle_missing='return_nan', handle_unknown='return_nan'),
    # ce.TargetEncoder(cols=encode_features, handle_missing='return_nan', handle_unknown='return_nan'),
    SimpleImputer(),
    # IterativeImputer(sample_posterior=True, random_state=42),
    # DecisionTreeClassifier(min_samples_leaf=250, max_depth=10, random_state=42)
)

pipeline

CPU times: user 299 µs, sys: 0 ns, total: 299 µs
Wall time: 306 µs


In [0]:
type(pipeline)

sklearn.pipeline.Pipeline